## DFT Screening - Construction of the chemical space

**Author:** Quentin Duez

This notebook generates files for DFT calculations and parses relevant chemical descriptors from the DFT results

**Outputs:** DFT input files, 'global_data.xlsx' and 'atom_data.xlsx' after parsing descriptors from the DFT *.log files

**Disclaimer :**
Parts of this notebook correspond to code available on the auto-qchem GitHub repository 

Please refer to the [original repository](https://github.com/doyle-lab-ucla/auto-qchem/) and [related publication](https://pubs.rsc.org/en/content/articlelanding/2022/re/d2re00030j#!divCitation)

In [3]:
import glob as glob
import logging
import os


from autoqchem.gaussian_input_generator import gaussian_input_generator
from autoqchem.gaussian_log_extractor import *
from autoqchem.molecule import molecule
from autoqchem.rdkit_utils import *

logging.basicConfig(level=logging.INFO)

RMSD_threshold = 0.35
k_in_kcal_per_mol_K = 0.0019872041
Hartree_in_kcal_per_mol = 627.5
T = 298

### Read diamine list and prepare files for DFT calculations

In [2]:
batch_df = pd.read_excel("Diamines_batch_all.xlsx")
smiles_str_list = batch_df["SMILES"].values

In [4]:
# Check if all SMILES are unique

def allUnique(x):
    seen = list()
    return not any(i in seen or seen.append(i) for i in x)


print(allUnique(smiles_str_list))

True


In [5]:
# Generate up to 5 conformers per compound
mols = [molecule(s, num_conf=5) for s in smiles_str_list]

[15:35:56] WARNING: Charges were rearranged



### Preparing files for DFT calculations

In [7]:
def gaussprep(
    molecule,
    workflow_type="equilibrium",
    theory="APFD",
    solvent="None",
    light_basis_set="6-31G*",
    heavy_basis_set="LANL2DZ",
    generic_basis_set="genecp",
    max_light_atomic_number=36,
    wall_time="23:59:00",
):
    molecule_workdir = os.path.join(molecule.inchikey)
    gig = gaussian_input_generator(
        molecule,
        workflow_type,
        molecule_workdir,
        theory,
        solvent,
        light_basis_set,
        heavy_basis_set,
        generic_basis_set,
        max_light_atomic_number,
    )

    gig.create_gaussian_files()


for mol in mols:
    gaussprep(mol, theory="B3LYP", light_basis_set="6-31G**", solvent="water")

INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 5 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 2 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 2 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 2 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 1 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 2 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 4 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 2 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 3 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input files for 4 conformations.
INFO:autoqchem.gaussian_input_generator:Generating Gaussian input file

### Store all the DFT folders in a folder called DFT_Data & run the DFT calculations

### Parsing the DFT *.log files to collect descriptors

In [9]:
def rdmol_from_mol(mol, postDFT=True) -> Chem.Mol:
    mol_name = mol.inchikey
    mol_name = mol_name[
        :14
    ]  # We only take the first block of the inchikey because the second block might change (rdkit rearranges the charges)
    elements, connectivity_matrix, charges = (
        mol.elements,
        mol.connectivity_matrix,
        mol.charges,
    )
    conformer_coordinates = []
    energies = []
    loglist = glob.glob(f"DFT_Data/{mol_name}*/*.log")
    keys = [*range(0, len(loglist), 1)]
    print(keys)
    for logfile in loglist:
        # print(logfile)
        le = gaussian_log_extractor(logfile)
        le.check_for_exceptions()
        le.get_atom_labels()
        # verify that the labels are in the same order in gaussian after running it
        labels_ok = True
        if tuple(le.labels) != tuple(elements):
            labels_ok = False
        le.get_geometry()
        conformer_coordinates.append(le.geom[list("XYZ")].values)
        le.get_descriptors()
        energies.append(le.descriptors["G"] * Hartree_in_kcal_per_mol)

    rdmol = get_rdkit_mol(elements, conformer_coordinates, connectivity_matrix, charges)
    return rdmol, energies, keys

In [10]:
global_df = pd.DataFrame()
atomic_df = pd.DataFrame()
for mol in mols:
    mol_data = {
        "can": mol.can,
        "inchi": mol.inchi,
        "inchikey": mol.inchikey,
        "elements": mol.elements,
        "charges": mol.charges.tolist(),
        "connectivity_matrix": mol.connectivity_matrix.flatten().tolist(),
    }
    print(mol.can, mol.inchikey)
    rdmol, energies, keys = rdmol_from_mol(mol)
    keep = prune_rmsds(rdmol, RMSD_threshold)
    logger.info(
        f"Molecule {mol.inchikey} has {len(keys) - len(keep)} / {len(keys)} duplicate conformers."
    )
    mol_name = mol.inchikey
    mol_name = mol_name[:14]

    # loop over the conformers and extract info
    conformations = []
    loglist = glob.glob(f"DFT_Data/{mol_name}*/*.log")
    for conf in loglist:
        index = loglist.index(conf)
        # Check if the conformer number is in the "keep" list
        if index in keep:
            # extract descriptors for this conformer from log file
            log = glob.glob(f"DFT_Data/{mol_name}*/{mol_name}*_conf_{index}.log")
            le = gaussian_log_extractor(log[0])
            # add descriptors to conformations list
            conformations.append(le.get_descriptors())
        else:
            continue

    # compute weights
    free_energies = np.array(
        [Hartree_in_kcal_per_mol * c["descriptors"]["G"] for c in conformations]
    )  # in kcal_mol
    free_energies -= free_energies.min()  # to avoid huge exponentials
    weights = np.exp(-free_energies / (k_in_kcal_per_mol_K * T))
    weights /= weights.sum()

    mol_global_df = pd.DataFrame()
    mol_atomic_df = pd.DataFrame()

    for c in conformations:
        index = conformations.index(c)
        global_data = {
            "can": mol_data["can"],
            "inchi": mol_data["inchi"],
            "inchi_key": mol_data["inchikey"],
            "index": keys[index],
            "weight": weights[index],
        }
        global_data.update(c["descriptors"])
        temp_df = pd.DataFrame.from_dict(global_data, orient="index").T
        mol_global_df = pd.concat([mol_global_df, temp_df], ignore_index=True)
        atomic_data = {
            "can": mol_data["can"],
            "inchi": mol_data["inchi"],
            "inchi_key": mol_data["inchikey"],
            "elements": mol_data["elements"],
            "connectivity_matrix": mol_data["connectivity_matrix"],
            "index": keys[index],
            "weight": weights[index],
        }
        atomic_data.update(c["atom_descriptors"])
        temp_df = pd.DataFrame.from_dict(atomic_data, orient="index").T
        mol_atomic_df = pd.concat([mol_atomic_df, temp_df], ignore_index=True)

    global_df = pd.concat([global_df, mol_global_df], ignore_index=True)
    atomic_df = pd.concat([atomic_df, mol_atomic_df], ignore_index=True)

global_df.to_excel("global_data.xlsx")
atomic_df.to_excel("atom_data.xlsx")

NCCCCCCCCCCCCN QFTYSVGGYOXFRQ-UHFFFAOYSA-N
[0, 1, 2, 3, 4]


INFO:autoqchem.gaussian_log_extractor:Molecule QFTYSVGGYOXFRQ-UHFFFAOYSA-N has 0 / 5 duplicate conformers.


NC1C(N)CCCC1 SSJXIUAHEKJCMH-WDSKDSINSA-N
[0, 1]


INFO:autoqchem.gaussian_log_extractor:Molecule SSJXIUAHEKJCMH-WDSKDSINSA-N has 0 / 2 duplicate conformers.


NCCCN XFNJVJPLKCPIBV-UHFFFAOYSA-N
[0, 1, 2]


...

INFO:autoqchem.gaussian_log_extractor:Molecule WUPRYUDHUFLKFL-UHFFFAOYSA-N has 3 / 5 duplicate conformers.


NC1=CC=C(OC2=CC=C(OC3=CC=C(N)C=C3C(F)(F)F)C=C2)C(C(F)(F)F)=C1 LACZRKUWKHQVKS-UHFFFAOYSA-N
[0, 1, 2, 3, 4]


INFO:autoqchem.gaussian_log_extractor:Molecule LACZRKUWKHQVKS-UHFFFAOYSA-N has 3 / 5 duplicate conformers.


NC1=CC=CC=C1NC2=CC=CC=C2 NFCPRRWCTNLGSN-UHFFFAOYSA-N
[0, 1]


INFO:autoqchem.gaussian_log_extractor:Molecule NFCPRRWCTNLGSN-UHFFFAOYSA-N has 0 / 2 duplicate conformers.
